## Scrapper

### 1. Scrapper

Only change:
1. file_url: year and quarter
2. csv file name : year_Q

Note: work on putting the scrapper in a function that takes year and quarter as parameters

CREDITS TO SIGMA CODING (https://github.com/areed1192/sigma_coding_youtube) FOR THE SCRAPPER PART OF THE CODE

In [ ]:
# # Import libraries
# import requests
# import urllib
# from bs4 import BeautifulSoup
# import json
# import pandas as pd
# import re

# # Define a master file url
# file_url = r"https://www.sec.gov/Archives/edgar/full-index/2005/QTR3/master.idx" # Full index
# #file_url = r"https://www.sec.gov/Archives/edgar/daily-index/2006/QTR2/master.20060627.idx" # Daily Index

# # make a request for the file above
# content = requests.get(file_url).content

# # write content to a text file
# with open('master_20190401.txt', 'wb') as f:
#     f.write(content)

# # Reading the content to a text file
# with open('master_20190401.txt', 'rb') as f:
#     byte_data = f.read()

# # Decode byte data
# data = byte_data.decode('utf-8').split('  ')

# # find the starting index
# for index, item in enumerate(data):

#     if 'ftp://ftp.sec.gov/edgar/' in item:
#         start_ind = index

# # create a new list that removes unecessary items
# data_format = data[start_ind + 1:]

# master_data = []

# #loop through the data list
# for index, item in enumerate(data_format):

#     if index == 0:
#         clean_item_data = item.replace('\n', '|').split('|')
#         clean_item_data = clean_item_data[8:]
#     else:
#         clean_item_data = item.replace('\n', '|').split('|')

#     for index, row in enumerate(clean_item_data):

#         #when you find the .txt file
#         if '.txt' in row:

#             mini_list = clean_item_data[(index - 4): index + 1]

#             if len(mini_list) != 0:
#                 mini_list[4] = "https://www.sec.gov/Archives/" + mini_list[4]
#                 master_data.append(mini_list)

# #master_data[:3]

# # Loop through the master data set
# for index, document in enumerate(master_data):

#     # create a dictionary
#     document_dict = {}
#     document_dict['cik_number'] = document[0]
#     document_dict['company_name'] = document[1]
#     document_dict['form_id'] = document[2]
#     document_dict['date'] = document[3]
#     document_dict['file_url'] = document[4].replace('-','') # This will remove the '-' from the url

#     master_data[index] = document_dict

# urls = []
# # Collect url of interest from the master dataset
# for document_dict in master_data:
#     if document_dict['form_id'] == 'DEF 14A': #Note: form id is case sentitive
#         #print(document_dict['company_name'])
#         target_url = document_dict['file_url']
#         target_url = target_url[:-4] + '/index.json'
#         #print(target_url)
#         urls.append(target_url)

# #print(urls)

# # redefine base url 
# base_url = r"https://www.sec.gov"
# Def14aLinks = []
# #loop through urls to get only the DEF 14A url
# for url in urls:
#     content = requests.get(url).json()

#     for file in content['directory']['item']:
#         if "14a" in file['name'] and file['name'].endswith(('htm', 'html')):
#             htm_summary = base_url + content['directory']['name'] + '/' + file['name']

# #             print('-'*100)
# #             print('File name:' + file['name'])
# #             print('File path:' + htm_summary)
# #             print(htm_summary)
#             Def14aLinks.append(htm_summary)

# print(Def14aLinks[0:5])          
# data_links = pd.DataFrame(Def14aLinks)
# data_links.to_csv('CSV FILE PATH HERE', index = False)


### 2. Parsing and creating the list of dataframes (called "tables")

In [ ]:
# Creating the list of dataframes, called "tables"

# Import packages
import requests
import urllib
from bs4 import BeautifulSoup
import json
import pandas as pd
import re

with open('YOUR_DATA_FILE.csv') as fp:
    def14aLinks = [filename.strip() for filename in fp.readlines()]
    def14aLinks = def14aLinks[1:]
    
tables = []
for link in def14aLinks:
    
    content = requests.get(link)
    soup = BeautifulSoup(content.text, 'lxml')

    try:
        search_result = pd.read_html(link, match="Beneficial", header=0)
        for df in search_result:
            df = df.astype('str')
            for column in df.columns:
                if df[column].str.lower().str.contains("%").any():
                    tables.append(df)
                    break
    except ValueError:
        continue

    # Find and add company name
    try:
        company = soup.select_one("p:contains(Name)").find_previous('p').text.strip()
        df['Company Name'] = company
    except AttributeError:
        pass

    # 5. Adding Date
    try:
        date = soup.find('p', text = re.compile(r"(?:January|February|March|April|May|June|July|August|September|October|November|December)")).text.strip()
        df['Filing Date'] = date
    except AttributeError:
        pass
    #print(df)
    
    #NOTE: THIS IS THE LONGEST PART OF THE CODE. MAY TAKE UP TO 4 HOURS, DEPENDING ON NUMBER OF ITERATIONS

## 3. Saving each dataframe in "tables" in an individual csv file

In [66]:
# Function to save each table to individual csv file
def individual_saver(t, year):
    #index count
    i = 0
    while True:
        try:
            temp_var = t[i]
            temp_var.to_csv('YOUR_PATH_HERE/Raw_csv/{}/sample{}.csv'.format(year, i))
            i += 1
        except IndexError:
            break

individual_saver(tables, 1998)

### 4. Tidying every dataframes in "tables" and appending result to initially created csv (See "Def 14A Final _ First CSV" notebook)

In [67]:
# Tidying and appending every subsequent dataframe to initial CSV
def tidy_saver(t):
    #index count
    i = 0
    while True:
        try:
            temp_var = t[i]
            temp_var = pd.DataFrame(temp_var)
            temp_var.columns = temp_var.iloc[0]
            temp_var = temp_var.drop(temp_var.index[0])
            try:
                temp_var = temp_var.drop(['nan'], axis = 1)
            except KeyError:
                pass
            column_names = ['Beneficial_Owner', 'share','Shares voting power', 'percent','Percent shares', 'Company name', 'Date filed']
            if len(temp_var.columns) == 7:
                temp_var.columns = column_names
                #temp_var = temp_var.drop(['share','percent'], axis = 1) #dropping these may not be ideal
                # as some values could land there. So, just clean them manually later on
            else:
                pass
            temp_var.to_csv('YOUR_DATA_FILE.csv', mode='a', header=False)
            # Note: while header = False, the index counting restarts at every new table appended.
            # So, use index counting to identify where to start editing a table
            i += 1
        except IndexError:
            break

tidy_saver(tables)